<a href="https://colab.research.google.com/github/HajjoujiProf/CUDA-Programming/blob/main/MyCUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64


--2021-02-24 10:09:01--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64 [following]
--2021-02-24 10:09:01--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64.deb?2zsAM9g3wNp-xDnrONksggsVoKBMPpcNzMb8HJ9_45ZUc8nosF4sWTDr_8JoZFzUyR8gSAwrzKx-boCcyW9NxvZEmndniRSelCchl7CUrcvDjgTiD

In [ ]:
!nvcc --version 


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:

!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-6v6sjzh4
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-6v6sjzh4
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=a26a58b00fcf45313236f8654095d4e912458090041b756a3884cf1e5124ab7f
  Stored in directory: /tmp/pip-ephem-wheel-cache-9t5vcln1/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:

%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}
int main() {
int a, b, c;
// host copies of variables a, b & c
int *d_a, *d_b, *d_c;
// device copies of variables a, b & c
int size = sizeof(int);
// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
// Setup input values  
c = 0;
a = 3;
b = 14;
// Copy inputs to device
cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
// Launch add() kernel on GPU
add<<<1,1>>>(d_a, d_b, d_c);
// Copy result back to host
cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf("result is %d\n",c);
// Cleanup
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}

result is 17



In [ ]:

%%cu
#include <stdio.h>
#include <stdlib.h>
#include<stdio.h>
#include<stdlib.h>

__global__ void print_from_gpu(void) {
    printf("Hello World! from thread [%d,%d] \
        From device\n", threadIdx.x,blockIdx.x);
}

int main(void) {
    printf("Hello World from host!\n");
    print_from_gpu<<<1,1>>>();
    cudaDeviceSynchronize();
    return 0;
}

Hello World from host!
Hello World! from thread [0,0]         From device



In [ ]:
%%cu
#include <stdio.h> 

int main() {
  int nDevices;

  cudaGetDeviceCount(&nDevices);
  for (int i = 0; i < nDevices; i++) {
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, i);
    printf("Device Number: %d\n", i);
    printf("  Device name: %s\n", prop.name);
    printf("  Memory Clock Rate (KHz): %d\n",
           prop.memoryClockRate);
    printf("  Memory Bus Width (bits): %d\n",
           prop.memoryBusWidth);
    printf("  Peak Memory Bandwidth (GB/s): %f\n\n",
           2.0*prop.memoryClockRate*(prop.memoryBusWidth/8)/1.0e6);
  }
}

Device Number: 0
  Device name: Tesla T4
  Memory Clock Rate (KHz): 5001000
  Memory Bus Width (bits): 256
  Peak Memory Bandwidth (GB/s): 320.064000




In [ ]:
    %%cu
    
    #include "cuda_runtime.h"
    #include "device_launch_parameters.h"
    #include <stdio.h>
    __global__ void HelloWorld()
    {
        printf("Hello world, %d, %d\n", blockIdx.x,
        threadIdx.x);
    }
int main()
    {
        HelloWorld << <2, 5 >> >();
    // хост ожидает завершения работы девайса
    //the host is waiting for the device to shutdown
        cudaDeviceSynchronize();
    // ожидаем нажатия любой клавиши
    // waiting for any key press
        getchar();
return 0; }

Hello world, 0, 0
Hello world, 0, 1
Hello world, 0, 2
Hello world, 0, 3
Hello world, 0, 4
Hello world, 1, 0
Hello world, 1, 1
Hello world, 1, 2
Hello world, 1, 3
Hello world, 1, 4



In [ ]:

%%cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>

// ядро
// core

__global__ void add( int *a, int *b, int *c ) {
*c = *a + *b; }

//главная функция
// Main Function

int main()
{
    // переменные на CPU
    // variables on the CPU
 
int a, b, c;
 
    // переменные на GPU
    // variables on the GPU
 
    int *dev_a, *dev_b, *dev_c;
    int size = sizeof( int );
//размерность
 // dimension

 // выделяем память на GPU
 // allocate memory to the GPU

cudaMalloc( (void**)&dev_a, size );
cudaMalloc( (void**)&dev_b, size );
cudaMalloc( (void**)&dev_c, size );
 
// инициализация переменных
 // variable initialization

a = 2;
b = 10;
 
// копирование информации с CPU на GPU
 // copying information from CPU to GPU

cudaMemcpy( dev_a, &a, size, cudaMemcpyHostToDevice );
cudaMemcpy( dev_b, &b, size, cudaMemcpyHostToDevice );
 
 // вызов ядра
 // kernel call

add<<< 1, 1 >>>( dev_a, dev_b, dev_c );
 
// копирование результата работы ядра с GPU на CPU
 // copying the result of the kernel from the GPU to the CPU
 
 cudaMemcpy( &c, dev_c, size, cudaMemcpyDeviceToHost );

    // вывод информации
    // information output
 
    printf("%d + %d = %d\n", a, b, c);
 
    // очищение памяти на GPU
    // clearing memory on GPU
 
    cudaFree( dev_a );
    cudaFree( dev_b );
    cudaFree( dev_c );
    return 0;
}



2 + 10 = 12



In [ ]:

%%cu
#include <stdio.h>
#include <assert.h>

// Convenience function for checking CUDA runtime API results
// can be wrapped around any runtime API call. No-op in release builds.
inline
cudaError_t checkCuda(cudaError_t result)
{
#if defined(DEBUG) || defined(_DEBUG)
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", 
            cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
#endif
  return result;
}

void profileCopies(float        *h_a, 
                   float        *h_b, 
                   float        *d, 
                   unsigned int  n,
                   char         *desc)
{
  printf("\n%s transfers\n", desc);

  unsigned int bytes = n * sizeof(float);

  // events for timing
  cudaEvent_t startEvent, stopEvent; 

  checkCuda( cudaEventCreate(&startEvent) );
  checkCuda( cudaEventCreate(&stopEvent) );

  checkCuda( cudaEventRecord(startEvent, 0) );
  checkCuda( cudaMemcpy(d, h_a, bytes, cudaMemcpyHostToDevice) );
  checkCuda( cudaEventRecord(stopEvent, 0) );
  checkCuda( cudaEventSynchronize(stopEvent) );

  float time;
  checkCuda( cudaEventElapsedTime(&time, startEvent, stopEvent) );
  printf("  Host to Device bandwidth (GB/s): %f\n", bytes * 1e-6 / time);

  checkCuda( cudaEventRecord(startEvent, 0) );
  checkCuda( cudaMemcpy(h_b, d, bytes, cudaMemcpyDeviceToHost) );
  checkCuda( cudaEventRecord(stopEvent, 0) );
  checkCuda( cudaEventSynchronize(stopEvent) );

  checkCuda( cudaEventElapsedTime(&time, startEvent, stopEvent) );
  printf("  Device to Host bandwidth (GB/s): %f\n", bytes * 1e-6 / time);

  for (int i = 0; i < n; ++i) {
    if (h_a[i] != h_b[i]) {
      printf("*** %s transfers failed ***\n", desc);
      break;
    }
  }

  // clean up events
  checkCuda( cudaEventDestroy(startEvent) );
  checkCuda( cudaEventDestroy(stopEvent) );
}

int main()
{
  unsigned int nElements = 4*1024*1024;
  const unsigned int bytes = nElements * sizeof(float);

  // host arrays
  float *h_aPageable, *h_bPageable;   
  float *h_aPinned, *h_bPinned;

  // device array
  float *d_a;

  // allocate and initialize
  h_aPageable = (float*)malloc(bytes);                    // host pageable
  h_bPageable = (float*)malloc(bytes);                    // host pageable
  checkCuda( cudaMallocHost((void**)&h_aPinned, bytes) ); // host pinned
  checkCuda( cudaMallocHost((void**)&h_bPinned, bytes) ); // host pinned
  checkCuda( cudaMalloc((void**)&d_a, bytes) );           // device

  for (int i = 0; i < nElements; ++i) h_aPageable[i] = i;      
  memcpy(h_aPinned, h_aPageable, bytes);
  memset(h_bPageable, 0, bytes);
  memset(h_bPinned, 0, bytes);

  // output device info and transfer size
  cudaDeviceProp prop;
  checkCuda( cudaGetDeviceProperties(&prop, 0) );

  printf("\nDevice: %s\n", prop.name);
  printf("Transfer size (MB): %d\n", bytes / (1024 * 1024));

  // perform copies and report bandwidth
  profileCopies(h_aPageable, h_bPageable, d_a, nElements, "Pageable");
  profileCopies(h_aPinned, h_bPinned, d_a, nElements, "Pinned");

  printf("n");

  // cleanup
  cudaFree(d_a);
  cudaFreeHost(h_aPinned);
  cudaFreeHost(h_bPinned);
  free(h_aPageable);
  free(h_bPageable);

  return 0;
}


Device: Tesla T4
Transfer size (MB): 16

Pageable transfers
  Host to Device bandwidth (GB/s): 4.856452
  Device to Host bandwidth (GB/s): 5.020136

Pinned transfers
  Host to Device bandwidth (GB/s): 12.165301
  Device to Host bandwidth (GB/s): 13.002207
n


Программа CUDA состоит из двух основных компонентов: хоста и ядра 
графического процессора.
Глобальная память - это память с высокой задержкой
 cudaMalloc() 

In [ ]:
%%cu

#include <stdio.h>
#include <cuda.h>
#include <math.h> 
#include <curand.h>
#include <curand_kernel.h>

#define NUM_BLOCK  1024
#define NUM_THREAD 1024

#define CUDA_CALL(x) do { if ((x) != cudaSuccess ) { printf (" Error at %s :%d\n - %s" , __FILE__ , __LINE__ , cudaGetErrorString(cudaGetLastError())) ; return EXIT_FAILURE ;}} while (0)
#define CURAND_CALL(x) do { if ((x) != CURAND_STATUS_SUCCESS ) { printf (" Error at %s :%d\n - %s" , __FILE__ , __LINE__, cudaGetErrorString(cudaGetLastError())) ; return EXIT_FAILURE ;}} while (0)

__global__ void checkIsInCircle(int *sum, float *randomX, float *randomY) {

	int id = blockIdx.x * NUM_BLOCK + threadIdx.x;

	float px = randomX[id];
	float py = randomY[id];

	if (px * px + py * py <= 1) {
		atomicAdd(&sum[blockIdx.x], 1);	 	
	}
}


int main(void) {
	int *sumHost, *sumDev;
	size_t sumSize = NUM_BLOCK * sizeof(int);
	CUDA_CALL(cudaMalloc((void **) &sumDev, sumSize));
	CUDA_CALL(cudaMemset(sumDev, 0, sumSize));
	sumHost = (int *)malloc(sumSize);

	float *randomX, *randomY;
	size_t randomSize = NUM_BLOCK * NUM_THREAD * sizeof(float);
	CUDA_CALL(cudaMalloc((void **) &randomX, randomSize));
	CUDA_CALL(cudaMalloc((void **) &randomY, randomSize));
		
	curandGenerator_t generator;

	CURAND_CALL(curandCreateGenerator(&generator, CURAND_RNG_PSEUDO_DEFAULT)); 

	cudaEvent_t start, stop;
	float mean_time = 0;
	float elapsedTime;

	int *allSum = (int *)malloc(NUM_BLOCK * sizeof(int));
	memset(allSum, 0, NUM_BLOCK * sizeof(int));

	int iterationsCount = 1000;

	for (int i = 0; i < iterationsCount; ++i)
	{
		cudaEventCreate(&start);
		cudaEventCreate(&stop);
		cudaEventRecord(start, 0);


		CURAND_CALL(curandGenerateUniform(generator, randomX, NUM_BLOCK * NUM_THREAD));
		CURAND_CALL(curandGenerateUniform(generator, randomY, NUM_BLOCK * NUM_THREAD)); 

		checkIsInCircle <<<NUM_BLOCK, NUM_THREAD>>> (sumDev, randomX, randomY); 
	
		CUDA_CALL(cudaMemcpy(sumHost, sumDev, sumSize, cudaMemcpyDeviceToHost));

		cudaEventRecord(stop, 0);
		cudaEventSynchronize(stop);
		cudaEventElapsedTime(&elapsedTime, start, stop);
		mean_time += elapsedTime;


		for (int i = 0; i < NUM_BLOCK; ++i)
		{
			allSum[i] += sumHost[i];
		}

		CUDA_CALL(cudaMemset(sumDev, 0, sumSize));

		cudaEventDestroy(start);
		cudaEventDestroy(stop);
	}

	printf("\nResult %fms\n", mean_time / iterationsCount);

	float count = 0;
	for(int tid = 0; tid < NUM_BLOCK; tid++) {
		count += (float) allSum[tid] / iterationsCount;
	}

	printf("\n%f vs %d", count, NUM_BLOCK * NUM_THREAD);
	
	float pi = (count / (NUM_BLOCK * NUM_THREAD)) * 4;

	printf("\nPI = %f\n", pi);


	free(sumHost); 
	cudaFree(sumDev);
	cudaFree(randomX);
	cudaFree(randomY);

	return 0;
}

/tmp/tmpxft_000000c0_00000000-10_ea559df7-6a2a-4dc2-9509-0029fba945ae.o: In function `main':
tmpxft_000000c0_00000000-5_ea559df7-6a2a-4dc2-9509-0029fba945ae.cudafe1.cpp:(.text+0x1a7): undefined reference to `curandCreateGenerator'
tmpxft_000000c0_00000000-5_ea559df7-6a2a-4dc2-9509-0029fba945ae.cudafe1.cpp:(.text+0x277): undefined reference to `curandGenerateUniform'
tmpxft_000000c0_00000000-5_ea559df7-6a2a-4dc2-9509-0029fba945ae.cudafe1.cpp:(.text+0x2ce): undefined reference to `curandGenerateUniform'
collect2: error: ld returned 1 exit status



In [ ]:


%%cu
#include <stdio.h>




//Elemetwise multiplicaton of two vectors // 
__global__ void kMartixByMatrixElementwise(const int nThreads, const float *m1, const float *m2, float *output) {
    /*  Computes the product of two arrays (elementwise multiplication).
     Inputs:
     m1: array
     m2: array
     output: array,the results of the multiplication are to be stored here
    */
	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	  {
		output[i] = m1[i] * m2[i];
	  }
}


//Elemetwise multiplicaton of two vectors Device //
__device__ float* dMartixByMatrixElementwise(const float *m1, const float *m2, float *output, const int width, const int height){

	kMartixByMatrixElementwise <<< width, height >>> ( width * height, m1, m2, output );
    cudaDeviceSynchronize();
    return output;
}




//CUDA kernel which finds the difference between two matrices //

__global__ void kMartixSubstractMatrix(const int nThreads, const float *m1, const float *m2, float *output) {
    /*  Computes the (elementwise) difference between two arrays
     Inputs:
     m1: array
     m2: array
     output: array,the results of the computation are to be stored here
     */

	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	  {
		output[i] = m1[i] - m2[i];
	  }
}





//
__device__ float* dMartixSubstractMatrix(const float *m1, const float *m2, float *output, const int width, const int height){

	kMartixSubstractMatrix <<< width, height >>> ( width * height, m1, m2, output );
    cudaDeviceSynchronize();
    return output;
}





//sigmoid function definition //

__global__ void kSigmoid(const int nThreads, float const *input, float *output){
    /*  Computes the value of the sigmoid function f(x) = 1/(1 + e^-x).
     Inputs:
     input: array
     output: array, the results of the computation are to be stored here
    */

	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	  {
		output[i] = 1.0 / (1.0 + std::exp(-input[i]));
	  }
}

__device__ void dSigmoid(float const *input, float *output, const int height, const int width){

	kSigmoid <<< height, width >>> (height * width, input, output);
	cudaDeviceSynchronize();
}



//Finding the values of the hidden layer function //

__global__ void kSigmoid_d(const int nThreads, float const *input, float *output) {
	/*  Computes the value of the sigmoid function derivative f'(x) = f(x)(1 - f(x)),
	    where f(x) is sigmoid function.
	    Inputs:
	    input: array
	    output: array, the results of the computation are to be stored here:
	    		x(1 - x) for every element of the input matrix m1.
	*/

	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	  {
		output[i] = input[i] * (1 - input[i]);
	  }
}

__device__ float* dSigmoid_d(float const *input, float *output, const int rows, const int columns){
	kSigmoid_d <<< rows, columns >>> (rows*columns, input, output);
	cudaDeviceSynchronize();
	return output;
}



//matrix multiplication function definition //

__global__ void kDot(const int nThreads, const float *m1, const float *m2, float *output, const int m1_rows , const int m1_columns, const int m2_columns ){
	/*  Computes the product of two matrices: m1 x m2.
	   	Inputs:
	    m1: array, left matrix of size m1_rows x m1_columns
	    m2: array, right matrix of size m1_columns x m2_columns (the number of rows in the right matrix
	    must be equal to the number of the columns in the left one)
	    output: array, the results of the computation are to be stored here:
	    		m1 * m2, product of two arrays m1 and m2, a matrix of size m1_rows x m2_columns
	    m1_rows: int, number of rows in the left matrix m1
	    m1_columns: int, number of columns in the left matrix m1
	    m2_columns: int, number of columns in the right matrix m2
	*/

	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	{
	    int r = (int)i / m2_columns;
	    int c = i % m2_columns;
	    float t_output = 0.f;

	    for( int k = 0; k < m1_columns; ++k ) {
	        t_output += m1[ r * m1_columns + k ] * m2[ k * m2_columns + c ];
	    }

	    output[i] = t_output;
	}
}

__device__ float* dDot(const float *m1, const float *m2, float *output, const int m1_rows , const int m1_columns, const int m2_columns ){

	kDot <<< m1_rows, m2_columns >>> (m1_rows * m2_columns, m1, m2, output, m1_rows , m1_columns, m2_columns );
	cudaDeviceSynchronize();
	return output;
}

__global__ void kDot_m1_m2T(const int nThreads, const float *m1, const float *m2, float *output, const int m1_columns, const int m2_rows ){
	/*  Updates the output matrix with the product of two matrices: m1 and m2 transposed.
	   	Inputs:
	    m1: array, left matrix of size m1_rows x m1_columns
	    m2: array, right matrix of size m2_rows x m1_columns (m2 transposed will be of size m1_columns x m2_rows)
	    output: array, the results of the computation are to be stored here:
	    		m1 * m2, product of two arrays m1 and m2, a matrix of size m1_rows x m2_rows
	    m1_columns: int, number of columns in the left matrix m1
	    m2_rows: int, number of rows in the left matrix m2
	*/

	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	{
		int r = (int)i / m2_rows;
		int c = i % m2_rows;
		float t_output = 0.0;
		int id_T;

		for( int k = 0; k < m1_columns; ++k ) {
			id_T = c * m1_columns + k;
			t_output += m1[ r * m1_columns + k ] * m2[ id_T ];
		}

		output[i] = t_output;
	}
}




//subroutine that multiplies matrix by transposed matrix //

__device__ float* dDot_m1_m2T(const float *m1, const float *m2, float *output, const int m1_rows , const int m1_columns, const int m2_rows )
{
	kDot_m1_m2T <<< m1_rows, m2_rows >>> ( m1_rows * m2_rows, m1, m2, output, m1_columns, m2_rows );
	cudaDeviceSynchronize();
	return output;
}



// subroutine that multiplies matrix by transposed matrix kernel  //

__global__ void kDot_m1T_m2(const int nThreads, const float *m1, const float *m2, float *output, const int m1_rows,
							const int m1_columns, const int m2_columns ){
	/*  Increments the output matrix with the product of two matrices: m1 transposed and m2.
	   	Inputs:
	    m1: array, left matrix of size m1_rows x m1_columns (m1 transposed will be of size m1_columns x m1_rows)
	    m2: array, right matrix of size m1_rows x m2_columns
	    output: array, the results of the computation are to be stored here:
	    		m1 * m2, product of two arrays m1 and m2, a matrix of size m1_columns x m2_columns
	    m1_rows: int, number of rows in the left matrix m1
	    m1_columns: int, number of columns in the left matrix m1
	    m2_rows: int, number of rows in the left matrix m2
	*/

	for (int i = blockIdx.x * blockDim.x + threadIdx.x;
		 i < nThreads;
		 i += blockDim.x * gridDim.x)
	{
	    int r = (int)i / m2_columns;
	    int c = i % m2_columns;
	    int id_T;
	    float t_output = 0.0;

	    for( int k = 0; k < m1_rows; ++k ) {
	    	id_T = k * m1_columns + r;
	        t_output += m1[ id_T ] * m2[ k * m2_columns + c ];
	    }

	    output[i] += t_output;
	}
}




__device__ void dDot_m1T_m2(const float *m1, const float *m2, float *output, const int m1_height , const int m1_width, const int m2_width )
{
	kDot_m1T_m2 <<< m1_width, m2_width >>> (m1_width * m2_width, m1, m2, output, m1_height, m1_width, m2_width );
	cudaDeviceSynchronize();
}



__device__ void kPrintMatrix (const float* M, int h, int w) {
    /*  Prints out the input array as h x w matrix.
     Inputs:
     m: vector, matrix of size n_rows x n_columns
     h: int, number of rows in the matrix M
     w: int, number of columns in the matrix M
     */
	for (int i = 0; i < h; i++){
		for (int j = 0; j < w; j++){
			printf("%f  ", M[i*w+j]);
		}
		printf("\n");
	}
	printf("\n");
}





// The loop runs for 50 iterations (epochs) and fits the vector of attributes X to the vector of classes y //


__global__ void kFit(	const float* X, const int X_w, const int X_h,
						const float* y, const int y_w,
						float* l1, const int l1_w, float* l_1_d,
						float* pred, float* pred_d,
						float* W0,
						float* W1,
						float* buffer
						)
{
    
   
	for (unsigned i = 0; i < 50; ++i) {

        dSigmoid(dDot(X, W0, l1, X_h, X_w, l1_w), l1, X_h, l1_w);
        dSigmoid(dDot(l1, W1, pred, X_h, l1_w, y_w), pred, X_h, y_w);
        dMartixByMatrixElementwise(dMartixSubstractMatrix(y, pred, pred_d, X_h, y_w), dSigmoid_d(pred, buffer, X_h, y_w), pred_d, X_h, y_w );
        dMartixByMatrixElementwise(dDot_m1_m2T(pred_d, W1, l_1_d, X_h, y_w, l1_w), dSigmoid_d(l1, buffer, X_h, l1_w), l_1_d, X_h, l1_w);
        dDot_m1T_m2( l1, pred_d, W1, X_h, l1_w, y_w );
        dDot_m1T_m2( X, l_1_d, W0, X_h, X_w, l1_w );
    }
}




// A Layer //

int main(void){
    
    //h_ stands for hostss (CPU) //
    //d_ stands for device  ( GPU )//




// define hyperparameters //

	const int TRAINING_SIZE = 4;
	const int TRAINING_DIM = 4;
	const int L1_SIZE = 8;

	// X, the first 4 lines from Iris dataset
	float h_X[TRAINING_SIZE*TRAINING_DIM] = {	5.1, 3.5, 1.4, 0.2,
												4.9, 3.0, 1.4, 0.2,
												6.2, 3.4, 5.4, 2.3,
												5.9, 3.0, 5.1, 1.8 };



//define inpute size //
	const signed int X_size = sizeof(h_X);





//allocate space on GPU for input data //
	float *d_X;
	cudaMalloc(&d_X, X_size);
	cudaMemcpy(d_X, h_X, X_size, cudaMemcpyHostToDevice);



// we intilize the first wheight matrix randomly //
	//WEIGHTS_0
	const long signed int W0_size = L1_SIZE*TRAINING_DIM*sizeof(float);
	float *h_W0 = (float*)malloc(W0_size);
	for (int i = 0; i < L1_SIZE*TRAINING_DIM; i++){
	    h_W0[i] = 0.1 * (2.0*rand()/RAND_MAX-1.0);
	}



// we allocate space on GPU for the weights //

	float *d_W0;
	cudaMalloc(&d_W0, W0_size);
	cudaMemcpy(d_W0, h_W0, W0_size, cudaMemcpyHostToDevice);

  // layer first size //

	//LAYER_1, LAYER_1_DELTA AND BUFFER OF LAYER 1 SIZE
	const long signed int L1_size = L1_SIZE*TRAINING_SIZE*sizeof(float);


  //define layer 1 variables ( delta and buffer ) // 

	float* h_layer_1 = (float*)malloc(L1_size);
	float* h_layer_1_delta = (float*)malloc(L1_size);
	float* h_buffer = (float*)malloc(L1_size);


  // give layer 1 initial 0 values // 

	for (int i = 0; i < L1_SIZE*TRAINING_SIZE; i++){
	    h_layer_1[i] = 0.0;
	    h_buffer[i] = 0.0;
	    h_layer_1_delta[i] = 0.0;
	}



//allocate GPU space for Layer 1 // 
	float *d_layer_1;
	cudaMalloc(&d_layer_1, L1_size);
	cudaMemcpy(d_layer_1, h_layer_1, L1_size, cudaMemcpyHostToDevice);


//allocate GPU space for Layer 1 buffer // 
	float *d_buffer;
	cudaMalloc(&d_buffer, L1_size);
	cudaMemcpy(d_buffer, h_buffer, L1_size, cudaMemcpyHostToDevice);



//allocate GPU space for Layer 1 delta //
	float *d_layer_1_delta;
	cudaMalloc(&d_layer_1_delta, L1_size);
	cudaMemcpy(d_layer_1_delta, h_layer_1_delta, L1_size, cudaMemcpyHostToDevice);



//allocated weight randomly 1 //
	//WEIGHTS_1
	const long signed int W1_size = L1_SIZE*sizeof(float);
	float *h_W1 = (float*)malloc(W1_size);
	for (int i = 0; i < L1_SIZE; i++){
	    h_W1[i] = 0.1* (2.0*rand()/RAND_MAX-1.0);
	}

	float *d_W1;
	cudaMalloc(&d_W1, W1_size);
	cudaMemcpy(d_W1, h_W1, W1_size, cudaMemcpyHostToDevice);


  //allocate GPU space for weiht 1 //

	//Y
	float h_y[4] = {	0,
						0,
						1,
						1 };


   //allocate GPU for labels //         
	const signed int y_size = sizeof(h_y);
	float *d_y;
	cudaMalloc(&d_y, y_size);
	cudaMemcpy(d_y, h_y, y_size, cudaMemcpyHostToDevice);



// define the prediction and prediction delta //
	//PRED AND PRED_DELTA
	float* h_pred = (float*)malloc(y_size);
	float* h_pred_delta = (float*)malloc(y_size);
	for (int i = 0; i < TRAINING_SIZE; i++){
	    h_pred[i] = 0.0;
	    h_pred_delta[i] = 0.0;
	}


//allocate GPU space for prediction //
	float *d_pred;
	cudaMalloc(&d_pred, y_size);
	cudaMemcpy(d_pred, h_pred, y_size, cudaMemcpyHostToDevice);


//allocate GPU space for prediction delta // 
	float *d_pred_delta;
	cudaMalloc(&d_pred_delta, y_size);
	cudaMemcpy(d_pred_delta, h_pred_delta, y_size, cudaMemcpyHostToDevice);

	kFit <<< 1, 1 >>> (	d_X, TRAINING_DIM, TRAINING_SIZE,
						d_y, 1,
						d_layer_1, L1_SIZE, d_layer_1_delta,
						d_pred,
						d_pred_delta,
						d_W0,
						d_W1,
						d_buffer);

	cudaMemcpy(h_pred, d_pred, y_size, cudaMemcpyDeviceToHost);


//free GPU memory //
	cudaFree(d_pred);
	cudaFree(d_X);
	cudaFree(d_y);
	cudaFree(d_layer_1_delta);
	cudaFree(d_pred_delta);
	cudaFree(d_W0);
	cudaFree(d_W1);
	cudaFree(d_buffer);



  // free CPU Memory //

	free(h_layer_1_delta);
	free(h_pred_delta);
	free(h_W0);
	free(h_W1);
	free(h_buffer);

  //print all the predictions //

	for (int i = 0; i < TRAINING_SIZE; i++){
		printf("Prediction[%i] : %f True Value[%i] : %f Error[%i] : %f\n", i, h_pred[i], i, h_y[i], i, h_pred[i] - h_y[i]);
	}

	free(h_pred);
}

/tmp/tmpx_pz9wd8/c7e452ef-ea0c-44bc-94d4-b71d2245fa77.cu(26): error: calling a __global__ function("kMartixByMatrixElementwise") from a __device__ function("dMartixByMatrixElementwise") is only allowed on the compute_35 architecture or above

/tmp/tmpx_pz9wd8/c7e452ef-ea0c-44bc-94d4-b71d2245fa77.cu(59): error: calling a __global__ function("kMartixSubstractMatrix") from a __device__ function("dMartixSubstractMatrix") is only allowed on the compute_35 architecture or above

/tmp/tmpx_pz9wd8/c7e452ef-ea0c-44bc-94d4-b71d2245fa77.cu(87): error: calling a __global__ function("kSigmoid") from a __device__ function("dSigmoid") is only allowed on the compute_35 architecture or above

/tmp/tmpx_pz9wd8/c7e452ef-ea0c-44bc-94d4-b71d2245fa77.cu(113): error: calling a __global__ function("kSigmoid_d") from a __device__ function("dSigmoid_d") is only allowed on the compute_35 architecture or above

/tmp/tmpx_pz9wd8/c7e452ef-ea0c-44bc-94d4-b71d2245fa77.cu(153): error: calling a __global__ function("kD

In [ ]:


%%cu

#include <iostream>
#include <time.h>
#include <random>
#include <curand.h>
#include <math.h>
#include "kernels.cuh"



int main()
{
	unsigned int n = 256*256;
	unsigned int m = 20000;
	int *h_count;
	int *d_count;
	curandState *d_state;
	float pi;


	// allocate memory
	h_count = (int*)malloc(n*sizeof(int));
	cudaMalloc((void**)&d_count, n*sizeof(int));
	cudaMalloc((void**)&d_state, n*sizeof(curandState));
	cudaMemset(d_count, 0, sizeof(int));


	// set up timing stuff
	float gpu_elapsed_time;
	cudaEvent_t gpu_start, gpu_stop;
	cudaEventCreate(&gpu_start);
	cudaEventCreate(&gpu_stop);
	cudaEventRecord(gpu_start, 0);


	// set kernel
	dim3 gridSize = 256;
	dim3 blockSize = 256;
	setup_kernel<<< gridSize, blockSize>>>(d_state);


	// monti carlo kernel
	monti_carlo_pi_kernel<<<gridSize, blockSize>>>(d_state, d_count, m);


	// copy results back to the host
	cudaMemcpy(h_count, d_count, sizeof(int), cudaMemcpyDeviceToHost);
	cudaEventRecord(gpu_stop, 0);
	cudaEventSynchronize(gpu_stop);
	cudaEventElapsedTime(&gpu_elapsed_time, gpu_start, gpu_stop);
	cudaEventDestroy(gpu_start);
	cudaEventDestroy(gpu_stop);


	// display results and timings for gpu
	pi = *h_count*4.0/(n*m);
	std::cout<<"Approximate pi calculated on GPU is: "<<pi<<" and calculation took "<<gpu_elapsed_time<<std::endl;


	//  serial verion
	clock_t cpu_start = clock();
	std::default_random_engine generator;
	std::uniform_real_distribution<float> distribution(0, 1.0);
	int count = 0;
	for(unsigned int i=0;i<n;i++){
		int temp = 0;
		while(temp < m){
			float x = distribution(generator);
			float y = distribution(generator);
			float r = x*x + y*y;

			if(r <= 1){
				count++;
			}
			temp++; 
		}
	}
	clock_t cpu_stop = clock();
	pi = 4.0*count/(n*m);
	std::cout<<"Approximate pi calculated on CPU is: "<<pi<<" and calculation took "<<1000*(cpu_stop - cpu_start)/CLOCKS_PER_SEC<<std::endl;



	// delete memory
	free(h_count);
	cudaFree(d_count);
	cudaFree(d_state);
}



/tmp/tmpw0fgknq9/9bc83706-c9a8-43a6-b2db-b25388d65447.cu:7:10: fatal error: kernels.cuh: No such file or directory
 #include "kernels.cuh"
          ^~~~~~~~~~~~~
compilation terminated.

